In [1]:
'''
import pandas as pd
%time 
column_names = ["user_id", "item_id", "category_id", "behavior_type", "input_timestamp"]
df=pd.read_csv("../input/userdata/UserBehavior.csv", \
                names = ("user_id", "item_id", "category_id", "behavior_type", "input_timestamp")) 
'''

'\nimport pandas as pd\n%time \ncolumn_names = ["user_id", "item_id", "category_id", "behavior_type", "input_timestamp"]\ndf=pd.read_csv("../input/userdata/UserBehavior.csv",                 names = ("user_id", "item_id", "category_id", "behavior_type", "input_timestamp")) \n'

In [2]:
!pip install pyspark
!pip install findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=9fa7f6966ffa5bfc12434d3dceda28e414cc13b86ae6710ef0cb46fa3270c55d
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").enableHiveSupport().getOrCreate()
## spark = SparkSession.builder.appName("SparkSQLExampleApp") .getOrCreate()

spark.conf.set("spark.sql.legacy.createHiveTableByDefault", False)
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark.conf.set("spark.sql.session.timeZone", 'UTC+08:00') ## SET TIME ZONE 'Asia/Shanghai';

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/06 01:04:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
import time 
##  %time  next line 
##  %%time  whole cell 
from pyspark.sql.functions import *
from pyspark.sql.types import ByteType, IntegerType, DateType, StringType

In [5]:
df=spark.read.csv("../input/userdata/UserBehavior.csv") 
df.schema.names

['_c0', '_c1', '_c2', '_c3', '_c4']

## 1. Data Cleaning 

In [6]:
## 1.1 column name 
df = df.toDF("user_id", "item_id", "category_id", "behavior_type", "input_timestamp")

In [7]:
## 1.2 data format 
## Unix time - Epoch time - seconds passed since Epoch time 1970-01-01 00:00:00 UTC. 
## do not put show at the end 
df = df.withColumn("tt", from_unixtime("input_timestamp"))

In [8]:
df = df.withColumn("d", to_date("tt")).withColumn("h", hour("tt"))
## df.withColumn("hour", substring("timestamp", 0, 13)).show()

In [9]:
df = df.drop("input_timestamp", "tt") 

In [10]:
df.show(5)

+-------+-------+-----------+-------------+----------+---+
|user_id|item_id|category_id|behavior_type|         d|  h|
+-------+-------+-----------+-------------+----------+---+
|      1|2268318|    2520377|           pv|2017-11-25|  1|
|      1|2333346|    2520771|           pv|2017-11-25|  6|
|      1|2576651|     149192|           pv|2017-11-25|  9|
|      1|3830808|    4181361|           pv|2017-11-25| 15|
|      1|4365585|    2520377|           pv|2017-11-25| 15|
+-------+-------+-----------+-------------+----------+---+
only showing top 5 rows



In [11]:
## 1.2 Data Type
df = df.withColumn("user_id",col("user_id").cast("Integer")).withColumn("item_id",col("item_id").cast("Integer"))
df = df.withColumn("category_id",col("category_id").cast("Integer"))
df = df.withColumn("d",col("d").cast("Date")).withColumn("h", col("h").cast("Integer"))

In [12]:
df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- behavior_type: string (nullable = true)
 |-- d: date (nullable = true)
 |-- h: integer (nullable = true)



In [13]:
## 1.3 null value 

## df.filter(df.user_id.isNull() & df.item_id.isNull() & df.category_id.isNull() \
##         & df.category_id.isNull() & df.input_timestamp.isNull()).show()

## no null value 

In [14]:

## 1.4 repeated value 

## df.dropDuplicates()

## df.groupBy("user_id", "item_id", "category_id", "behavior_type", "input_timestamp").count()

## %time : long running time 
## explaintion: repeated with multiple devices


In [15]:
## df.count()
## 100150807

In [16]:
## 1.5 anomaly 

## df_anomaly_lower = df.filter(df.d < '2017-11-25')
## df_anomaly_higher = df.filter(df.d > '2017-12-03')

## df_anomaly_higher.count()
## 2428
## df_anomaly_lower.count()
## 53148

In [17]:
## higher_sample = df_anomaly_higher.sample(False, 0.1, seed=0)
## higher_sample
## lower_sample = df_anomaly_lower.sample(False, 0.1, seed=0)
## lower_sample

## sample with replacement (default False).

In [18]:
df = df.filter((df.d > '2017-11-24') & (df.d < '2017-12-04') )

## df.count()
## 100095231

In [19]:
## df.withColumn("row_num", row_number().over(Window.partitionBy("Group") 
## row_number(): only used on partitionBy
## groupBy vs partitionBy

## df.count()
## df.distinct().count()
## output00 = df.select(countDistinct("user_id", "item_id", "category_id", "behavior_type", "input_timestamp"))

## BIG DATA
## https://www.mikulskibartosz.name/how-to-speed-up-pyspark/
## spark.default.parallelism 

## 2. Data Analysis

In [20]:
df.registerTempTable("user_behavior")
## df.createOrReplaceTempView("user_behavior")

/opt/conda/lib/python3.7/site-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [21]:
df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- behavior_type: string (nullable = true)
 |-- d: date (nullable = true)
 |-- h: integer (nullable = true)



In [22]:
spark.sql(" CREATE TABLE temp_behavior (SELECT * FROM user_behavior LIMIT 5) ")
df_temp = spark.table("temp_behavior")
df_temp.printSchema()

22/07/06 01:04:14 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
22/07/06 01:04:14 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
22/07/06 01:04:19 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
22/07/06 01:04:19 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@172.19.2.2
22/07/06 01:04:19 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException


22/07/06 01:04:22 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
22/07/06 01:04:22 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
22/07/06 01:04:22 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
22/07/06 01:04:22 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
22/07/06 01:04:22 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- behavior_type: string (nullable = true)
 |-- d: date (nullable = true)
 |-- h: integer (nullable = true)



In [23]:
spark.sql(" DESCRIBE user_behavior ")

col_name,data_type,comment
user_id,int,null
item_id,int,null
category_id,int,null
behavior_type,string,null
d,date,null
h,int,null


### 0.1    overview 

In [24]:
## index = df.count()
## df_out = df.select(countDistinct("user_id")) 100095231
## df_out = df.select(countDistinct("item_id")) 4161138
## df_out = df.select(countDistinct("category_id")) 9437
## df.select(approx_count_distinct("item_id")).collect() 4117166
## df.select(countDistinct("user_id", "item_id", "category_id")) together 

In [25]:
## df_out

In [26]:
index = 100095231
user_count = 987991
item_count = 4161138
category_count = 9437

In [27]:
import pandas as pd 
import numpy as np

In [28]:
columns = ["index","user_count", "user_count", "user_count"]
data =np.array((index, user_count, item_count, category_count)).reshape((1,4))
df_output = pd.DataFrame(data, columns = columns)

In [29]:
df_output.to_csv('./01_overview.csv')

### 1.1 traffic_conversion 

In [30]:
spark.sql(" CREATE TABLE 11_traffic_conversion ( SELECT SUM(CASE WHEN behavior_type ='pv' THEN 1 ELSE 0 END) AS pv_count, \
SUM(CASE WHEN behavior_type ='fav' THEN 1 ELSE 0 END) AS fav_count, \
SUM(CASE WHEN behavior_type ='cart' THEN 1 ELSE 0 END) AS cart_count, \
SUM(CASE WHEN behavior_type ='buy' THEN 1 ELSE 0 END) AS buy_count \
FROM user_behavior ) ")

""


In [31]:
%%time
df_output = spark.table("11_traffic_conversion")
df_output

CPU times: user 0 ns, sys: 974 µs, total: 974 µs
Wall time: 49.2 ms


pv_count,fav_count,cart_count,buy_count
89660688,2888258,5530446,2015839


In [32]:
%%time
df_output.toPandas().to_csv('./11_traffic_conversion.csv')

CPU times: user 16.8 ms, sys: 5.02 ms, total: 21.8 ms
Wall time: 179 ms


### 1.2 user_conversion 

In [33]:
spark.sql(" CREATE TABLE 12_user_behavior_count ( SELECT user_id, \
SUM(CASE WHEN behavior_type ='pv' THEN 1 ELSE 0 END) AS pv_count, \
SUM(CASE WHEN behavior_type ='fav' THEN 1 ELSE 0 END) AS fav_count, \
SUM(CASE WHEN behavior_type ='cart' THEN 1 ELSE 0 END) AS cart_count, \
SUM(CASE WHEN behavior_type ='buy' THEN 1 ELSE 0 END) AS buy_count \
FROM user_behavior \
GROUP BY user_id) ")


""


In [34]:
spark.sql(" CREATE TABLE 12_user_conversion ( select SUM(CASE WHEN  pv_count > 0 THEN 1 ELSE 0 END) AS pv_user_count, \
SUM(CASE WHEN  fav_count > 0 THEN 1 ELSE 0 END) AS fav_user_count, \
SUM(CASE WHEN  cart_count > 0 THEN 1 ELSE 0 END) AS cart_user_count, \
SUM(CASE WHEN  buy_count > 0 THEN 1 ELSE 0 END) AS buy_user_count \
from 12_user_behavior_type ) ")

AnalysisException: Table or view not found: 12_user_behavior_type; line 1 pos 314;
'CreateTable `default`.`12_user_conversion`, ErrorIfExists
+- 'Project ['SUM(CASE WHEN ('pv_count > 0) THEN 1 ELSE 0 END) AS pv_user_count#282, 'SUM(CASE WHEN ('fav_count > 0) THEN 1 ELSE 0 END) AS fav_user_count#283, 'SUM(CASE WHEN ('cart_count > 0) THEN 1 ELSE 0 END) AS cart_user_count#284, 'SUM(CASE WHEN ('buy_count > 0) THEN 1 ELSE 0 END) AS buy_user_count#285]
   +- 'UnresolvedRelation [12_user_behavior_type], [], false


In [ ]:
df_output = spark.table("12_user_conversion")
df_output

In [ ]:
df_output.toPandas().to_csv('./12_user_conversion.csv')

### 2.1 user_in_days 

In [ ]:
spark.sql(" CREATE TABLE 21_user_in_days (SELECT d, \
COUNT(DISTINCT (user_id)) AS user_count, \
SUM(CASE WHEN behavior_type ='pv' THEN 1 ELSE 0 END) AS pv_count, \
SUM(CASE WHEN behavior_type ='fav' THEN 1 ELSE 0 END) AS fav_count, \
SUM(CASE WHEN behavior_type ='cart' THEN 1 ELSE 0 END) AS cart_count, \
SUM(CASE WHEN behavior_type ='buy' THEN 1 ELSE 0 END) AS buy_count \
FROM user_behavior \
GROUP BY d \
ORDER BY d )")

In [ ]:
df_output = spark.table("21_user_in_days")
df_output

In [ ]:
df_output.toPandas().to_csv('./21_user_in_days.csv')

### 2.2 user_in_hours 

In [ ]:
spark.sql(" CREATE TABLE 22_user_in_hours  (SELECT h, \
COUNT(DISTINCT (user_id)) AS user_count, \
SUM(CASE WHEN behavior_type ='pv' THEN 1 ELSE 0 END) AS pv_count, \
SUM(CASE WHEN behavior_type ='fav' THEN 1 ELSE 0 END) AS fav_count, \
SUM(CASE WHEN behavior_type ='cart' THEN 1 ELSE 0 END) AS cart_count, \
SUM(CASE WHEN behavior_type ='buy' THEN 1 ELSE 0 END) AS buy_count \
FROM user_behavior \
GROUP BY h \
ORDER BY h )")

In [ ]:
df_output = spark.table("22_user_in_hours")
df_output

In [ ]:
df_output.toPandas().to_csv('./22_user_in_hours.csv')

### 3.1 RFM Model

In [ ]:
spark.sql(" CREATE TABLE 31_RF (SELECT user_id, \
DATEDIFF('2017-12-03', max(d)) AS recency, \
COUNT(user_id) AS frequency \
FROM user_behavior \
WHERE behavior_type='buy' \
GROUP BY user_id \
ORDER BY recency) ")

In [ ]:
spark.sql(" CREATE TABLE 31_RF_rating (SELECT user_id, \
(CASE WHEN recency > 3 THEN 1 \
WHEN recency BETWEEN 2 and 3 THEN 2 \
WHEN recency BETWEEN 0 and 1 THEN 3 ELSE 0 END) AS R_rating, \
(CASE WHEN frequency BETWEEN 0 and 1 THEN 1 \
WHEN frequency BETWEEN 2 and 3 THEN 2 \
WHEN frequency >3 THEN 3 ELSE 0 END ) AS F_rating \
FROM 31_RF \
ORDER BY R_rating, F_rating) ")

In [ ]:
## df_test_RF = spark.sql(" select * from 31_RF limit 20 ")
## df_test_RF_rating = spark.sql(" select * from 31_RF_rating limit 20 ")
## df_test_RF
## df_test_RF_rating

In [ ]:
spark.sql(" CREATE TABLE 31_RF_caculation (SELECT R_rating, \
F_rating, \
COUNT(*) AS RF_count \
FROM 31_RF_rating \
GROUP BY R_rating, F_rating \
ORDER BY R_rating, F_rating ) ")

In [ ]:
df_output = spark.table("31_RF_caculation")
df_output

In [ ]:
df_output.toPandas().to_csv('./31_RF_caculation.csv')

In [ ]:
spark.sql(" CREATE TABLE 32_F_distribution (SELECT frequency, \
COUNT(frequency) AS user_count \
FROM 31_RF \
GROUP by frequency \
ORDER by frequency) ")

In [ ]:
df_output = spark.table("32_F_distribution")
df_output

In [ ]:
df_output.toPandas().to_csv('./32_F_distribution.csv')

In [ ]:
spark.sql(" CREATE TABLE 32_R_distribution (SELECT recency, \
COUNT(recency) AS user_count \
FROM 31_RF \
GROUP by recency \
ORDER by recency) ")

In [ ]:
df_output = spark.table("32_R_distribution")
df_output

In [ ]:
df_output.toPandas().to_csv('./32_R_distribution.csv')

### 4.1 Items (80/20 model)

In [ ]:
spark.sql(" CREATE TABLE 41_item_sale AS ( SELECT item_id,  \
COUNT(item_id) AS buy_count  \
FROM user_behavior  \
WHERE behavior_type = \"buy\"  \
GROUP BY item_id  \
ORDER BY buy_count) ")  \

In [ ]:
spark.sql(" CREATE TABLE 41_item_sales_distribution ( SELECT buy_count,  \
COUNT(buy_count) AS item_count \
FROM 41_item_sale \
GROUP by buy_count  \
ORDER by buy_count ) ")

In [ ]:
df_output = spark.table("41_item_sales_distribution")
df_output

In [ ]:
df_output.toPandas().to_csv('./41_item_sales_distribution.csv')

### 4.1 category

In [ ]:
### category sales distrub

In [ ]:
spark.sql(" CREATE TABLE 42_cat_sale  AS ( SELECT category_id,  \
COUNT(category_id) AS buy_count  \
FROM user_behavior  \
WHERE behavior_type = \"buy\"  \
GROUP BY category_id  \
ORDER BY buy_count) ")  \

In [ ]:
spark.sql(" CREATE TABLE 42_cat_sales_distribution  ( SELECT buy_count,  \
COUNT(buy_count) AS cat_count \
FROM 42_cat_sale \
GROUP by buy_count  \
ORDER by buy_count ) ")

In [ ]:
df_output = spark.table("42_cat_sales_distribution ")
df_output

In [ ]:
df_output.toPandas().to_csv('./42_cat_sales_distribution .csv')